# List of My Movies

In [187]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from combine import combine
from fsUtils import moveFile


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-01-05 10:52:51.380185


In [208]:
import re
from time import sleep
from collections import OrderedDict
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile
from fsUtils import setDir, isDir, mkDir, setFile, isFile, setSubFile, moveFile
from fileUtils import getBaseFilename, getDirname, getFileBasics
from searchUtils import findSubPatternExt, findPatternExt, findExt, findNearest
from strUtils import convertCurrency
from webUtils import getWebData, getHTML
from numpy import repeat
from movieDB import movieDB
from os import getcwd
from os.path import join
from glob import glob
import operator
from movieRenames import manualRenames

##############################################################################################################################
# Combine Movies
##############################################################################################################################
class mymovies(movieDB):
    def __init__(self, basedir=None):
        self.name = "mymovies"
        movieDB.__init__(self, dbdir=self.name)
        self.combine = None
        
    def setCombinedMovies(self, combine):
        self.combine = combine
        
    def getCombinedMovies(self, debug=False):        
        savename = setFile(self.combine.getResultsDir(), "movies.json")
        if not isFile(savename):
            raise ValueErrro("Cannot access {0}".format(savename))
        combinedMovies = getFile(savename)
        if debug:
            print("Found {0} combined movies".format(len(combinedMovies)))
        return combinedMovies
        
    def getMyMovies(self, debug=False): 
        savename = setFile(self.getDataDir(), "mymovies.json")
        if not isFile(savename):
            raise ValueErrro("Cannot access {0}".format(savename))
        mine = getFile(savename)
        if debug:
            print("Found {0} my movies".format(len(mine)))
        return mine
        
    def findMyMovies(self, debug=False):
        movies = glob("/Volumes/*/Movies/*")
        mine   = dict(zip([getBaseFilename(x) for x in movies], movies))
        print("Found {0} movies on my disks".format(len(movies)))
        savename = setFile(self.getDataDir(), "mymovies.json")
        saveFile(idata=mine, ifile=savename, debug=True)
        
        
    def searchMyMovies(self, movie, year, cutoff=0.8):
        moviesMine = self.getMyMovies()
        key = "{0} [{1}]".format(movie, year)
        result     = findNearest(key, moviesMine.keys(), 3, 0.95)
        if len(result) > 0:
            print("{0}  --->  {1}".format(movie, result))
            return
        else:
            key = "{0} [{1}]".format(movie, year)
            result = findNearest(key, moviesMine.keys(), 3, cutoff)
            if len(result) > 0:
                print("{0}  --->  {1}".format(movie, result))
                return
        print("No match found")
        return
    
    
    def suggestRenames(self):

        movies = glob("/Volumes/Download/MoviesFinished/*.*")
        for movie in movies:
            fileInfo = getFileBasics(movie)
            name = fileInfo[1]
                        
            ps=["1080", "720", "480"]
            for p in ps:
                name = name.replace("{0}p".format(p), "")

            xs=['264']
            for x in xs:
                name = name.replace("x{0}".format(x), "")
            
            ts=["BluRay", "BRrip", "WEBRip", "HDTVRip", "GAZ", "BrRip"]
            for t in ts:
                name = name.replace("{0}".format(t), "")
                
            vs=["-[YTS.AG]", "-[YTS.AM]", "YIFY", "[YTS.AG]"]
            for v in vs:
                name = name.replace("{0}".format(v), "")
                
            name = name.replace(".", " ")
            name=name.strip()

            dst = join(fileInfo[0], "".join([name, fileInfo[2]]))
            if name != fileInfo[1]:
                moveFile(src=movie, dst=dst, debug=True)


        movies = glob("/Volumes/Download/MoviesFinished/*.*")   
        for movie in movies:
            fileInfo = getFileBasics(movie)            
            name  = fileInfo[1][:-4].strip()
            year  = fileInfo[1][-4:]
            
            try:
                int(year)
                name = "{0} [{1}]".format(name, year)
            except:
                print("Cannot rename {0}".format(movie))
                continue
            
            src = movie
            dst = join(fileInfo[0], "".join([name, fileInfo[2]]))
            moveFile(src=src, dst=dst, debug=True)
            print("")
        
        
    def mergeMovies(self, toFile=False):
        moviesCombined = self.getCombinedMovies()
        moviesMine     = self.getMyMovies()
        renames        = {}
        filename = setFile(self.getResultsDir(), "mymovies.dat")

        if toFile:
            print("{0: <6}{1: <3}{2: <100}{3: <20}".format("Year", u'\u2713', "Movie", "Source"), file=open(filename, "w"))
        else:
            print("{0: <6}{1: <3}{2: <100}{3: <20}".format("Year", u'\u2713', "Movie", "Source"))

        pYear = None
        for key,name in moviesCombined.items():
            movie = key[:-7]
            year  = key[-5:-1]

            location = moviesMine.get(key)
            if location is None:
                for dYear in [-1,1,-2,2]:
                    dKey = "{0} [{1}]".format(movie, int(year)+dYear)
                    if moviesMine.get(dKey) is not None:
                        location = moviesMine[dKey]
                        break
            if location is None:
                result = findNearest(key, moviesMine.keys(), 1, 0.98)
                if len(result) > 0:
                    location = u'\u2248'
                    myMovie  = moviesMine[result[0]]
                    fileInfo = getFileBasics(myMovie)
                    myMovieYear = fileInfo[1][-5:-1]
                    if abs(int(year) - int(myMovieYear)) <= 0:
                        renames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))
                else:
                    result = findNearest(key, moviesMine.keys(), 1, 0.90)
                    if len(result) > 0:
                        location = u'\u2047'
                        myMovie  = moviesMine[result[0]]
                        fileInfo = getFileBasics(myMovie)
                        myMovieYear = fileInfo[1][-5:-1]
                        if abs(int(year) - int(myMovieYear)) <= 0:
                            renames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))
                    else:
                        location = ""
            else:
                location = u'\u2713'
                
            
            #print(pYear,year)
            if pYear is not None and pYear != year:
                print("\n\n")
            pYear = year
                
            if toFile:
                print("{0: <6}{1: <3}{2: <100}{3: <20}".format(year, location, movie, name), file=open(filename, "a"))
            else:
                print("{0: <6}{1: <3}{2: <100}{3: <20}".format(year, location, movie, name))

                
                
        print("\n\n\n")
        for key,rename in renames.items():
            if key != rename:
                print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(key, rename))

In [239]:
mine = mymovies()
mine.setCombinedMovies(combine())
_ = clock("Last Run")

Current Time is Sat Jan 05, 2019 16:28:03 for Last Run


In [210]:
mine.suggestRenames()

Cannot rename /Volumes/Download/MoviesFinished/The Principal [1987].mp4


In [237]:
mine.findMyMovies()
_ = clock("Last Run")

Found 5015 movies on my disks
Saving data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
Saved data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
  --> This file is 406.3kB.
Current Time is Sat Jan 05, 2019 16:27:24 for Last Run


In [216]:
mine.searchMyMovies("Dr. Strangelove", 1921, cutoff=0.7)

Dr. Strangelove  --->  ['Dr. Strangelove [1964]']


In [238]:
mine.mergeMovies()

Year  ✓  Movie                                                                                               Source              
1915  ✓  The Birth Of A Nation                                                                               Rotten Tomatoes     



1920     The Cabinet Of Dr. Caligari (Das Cabinet des Dr. Caligari)                                          Rotten Tomatoes     



1921  ✓  The Four Horsemen Of The Apocalypse                                                                 Wiki Film           
1921     The Kid                                                                                             Wiki Film           
1921  ✓  Orphans Of The Storm                                                                                Wiki Film           
1921  ⁇  The Three Musketeers                                                                                Wiki Film           
1921  ✓  The Sheik                                                                  

1926     My Yankee Senor                                                                                     Wiki Film           



1927  ✓  Metropolis                                                                                          Rotten Tomatoes     
1927     Sunrise A Song Of Two Humans                                                                        Rotten Tomatoes     
1927  ✓  The Jazz Singer                                                                                     Wiki Film           
1927     Wings                                                                                               Wiki Film           
1927     Blake Of Scotland Yard                                                                              Wiki Film           
1927     7th Heaven                                                                                          Wiki Film           
1927     Love                                                                          

1931     Reducing                                                                                            Wiki Film           
1931     Daddy Long Legs                                                                                     Wiki Film           



1932  ✓  Shanghai Express                                                                                    Ultimate Movie Rankings
1932  ✓  Grand Hotel                                                                                         Ultimate Movie Rankings
1932     A Farewell To Arms                                                                                  Ultimate Movie Rankings
1932     The Kid From Spain                                                                                  Ultimate Movie Rankings
1932  ✓  One Hour With You                                                                                   Ultimate Movie Rankings
1932     Horse Feathers                                                 

1936     Come And Get It                                                                                     Oscar               
1936  ✓  Modern Times                                                                                        Rotten Tomatoes     
1936     San Francisco                                                                                       Ultimate Movie Rankings
1936     Dodsworth                                                                                           Ultimate Movie Rankings
1936     Libeled Lady                                                                                        Ultimate Movie Rankings
1936     Three Smart Girls                                                                                   Ultimate Movie Rankings
1936  ✓  After The Thin Man                                                                                  Ultimate Movie Rankings
1936     Swing Time                                                        

1939     Gunga Din                                                                                           Wiki Film           
1939  ✓  Drums Along The Mohawk                                                                              Wiki Film           
1939     Stanley And Livingstone                                                                             Wiki Film           
1939     Union Pacific                                                                                       Wiki Film           
1939     Another Thin Man                                                                                    Wiki Film           
1939     The Old Maid                                                                                        Wiki Film           
1939     Hollywood Cavalcade                                                                                 Wiki Film           
1939     Destry Rides Again                                                               

...

2014     A Summer's Tale                                                                                     Rotten Tomatoes     
2014  ✓  Jodorowsky's Dune                                                                                   Rotten Tomatoes     
2014     Elaine Stritch Shoot Me                                                                             Rotten Tomatoes     
2014     Keep On Keepin' On                                                                                  Rotten Tomatoes     
2014  ✓  The Babadook                                                                                        Rotten Tomatoes     
2014  ✓  Leviathan                                                                                           Rotten Tomatoes     
2014  ✓  Life Itself                                                                                         Rotten Tomatoes     
2014  ✓  Mr. Turner                                                                       

2015     Finders Keepers                                                                                     Rotten Tomatoes     
2015     Welcome To Leith                                                                                    Rotten Tomatoes     
2015  ✓  Paddington                                                                                          Rotten Tomatoes     
2015  ✓  Timbuktu                                                                                            Rotten Tomatoes     
2015  ✓  Phoenix                                                                                             Rotten Tomatoes     
2015  ⁇  Ingrid Bergman In Her Own Words (Jag är Ingrid)                                                     Rotten Tomatoes     
2015  ✓  Court                                                                                               Rotten Tomatoes     
2015  ✓  Tangerine                                                                        

2016     The Age Of Shadows                                                                                  Rotten Tomatoes     
2016     Tower                                                                                               Rotten Tomatoes     
2016     Cameraperson                                                                                        Rotten Tomatoes     
2016     Under The Shadow                                                                                    Rotten Tomatoes     
2016     The Wailing (Goksung)                                                                               Rotten Tomatoes     
2016  ✓  Don't Think Twice                                                                                   Rotten Tomatoes     
2016     The Fits                                                                                            Rotten Tomatoes     
2016     Almost Holy (Crocodile Gennadiy)                                                 

2017     I, Tonya                                                                                            Oscar               
2017     Call Me By Your Name                                                                                Oscar               
2017  ✓  Get Out                                                                                             Oscar               
2017  ✓  Bright Lights Starring Carrie Fisher And Debbie Reynolds                                            Rotten Tomatoes     
2017     Strong Island                                                                                       Rotten Tomatoes     
2017     The Work                                                                                            Rotten Tomatoes     
2017  ⁇  The Happiest Day In The Life Of Olli Mäki (Hymyilevä mies)                                          Rotten Tomatoes     
2017     Dawson City Frozen Time                                                          

2017     It                                                                                                  Ultimate Movie Rankings
2017  ✓  Rogue One A Star Wars Story                                                                         Box Office Mojo     
2017  ≈  Star Wars: The Last Jedi                                                                            Box Office Mojo     
2017  ✓  Beauty And The Beast                                                                                Box Office Mojo     
2017     Spider-Man Homecoming                                                                               Box Office Mojo     
2017  ✓  Sing                                                                                                Box Office Mojo     
2017  ⁇  Despicable Me 3                                                                                     Box Office Mojo     
2017  ✓  Fantastic Beasts And Where To Find Them                                       

In [175]:
mine.searchMyMovies("Avengers: Age Of Ultron", 2014, cutoff=0.7)

Avengers: Age Of Ultron  --->  ['Avengers Age Of Ultron [2015]']


In [234]:

moveFile(src="/Volumes/Download/Movies/The Four Horsemen of the Apocalypse [1921].avi", dst="/Volumes/Download/Movies/The Four Horsemen Of The Apocalypse [1921].avi", debug=True)

moveFile(src="/Volumes/Download/Movies/Orphans of the Storm [1921].avi", dst="/Volumes/Download/Movies/Orphans Of The Storm [1921].avi", debug=True)


Moving /Volumes/Download/Movies/The Four Horsemen of the Apocalypse [1921].avi to /Volumes/Download/Movies/The Four Horsemen Of The Apocalypse [1921].avi
  Not moving because /Volumes/Download/Movies/The Four Horsemen Of The Apocalypse [1921].avi exists.
Moving /Volumes/Download/Movies/Orphans of the Storm [1921].avi to /Volumes/Download/Movies/Orphans Of The Storm [1921].avi
  Not moving because /Volumes/Download/Movies/Orphans Of The Storm [1921].avi exists.


In [240]:

moveFile(src="/Volumes/Download/Movies/A Farewell to Arms [1957].mp4", dst="/Volumes/Download/Movies/A Farewell To Arms [1957].mp4", debug=True)


Moving /Volumes/Download/Movies/A Farewell to Arms [1957].mp4 to /Volumes/Download/Movies/A Farewell To Arms [1957].mp4
  Not moving because /Volumes/Download/Movies/A Farewell To Arms [1957].mp4 exists.
